In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [4]:
data_ready = pd.read_csv("data_ready.csv")

In [5]:
data_ready[data_ready.select_dtypes(include=['float64']).columns] = data_ready.select_dtypes(include=['float64']).astype("float32")

In [6]:
# 70% training data, 15% validation, 15% test
train_dev, test = train_test_split(data_ready, test_size=0.15, random_state=42)
train, dev = train_test_split(train_dev, test_size=0.176, random_state=42)

In [7]:
# convert to tensors and extract labels

train_label = train.pop("5YR_SURV")
dev_label = dev.pop("5YR_SURV")
test_label = test.pop("5YR_SURV")

train_tf = tf.convert_to_tensor(train)
dev_tf = tf.convert_to_tensor(dev)
test_tf = tf.convert_to_tensor(test)

train_label_tf = tf.convert_to_tensor(train_label)
dev_label_tf = tf.convert_to_tensor(dev_label)
test_label_tf = tf.convert_to_tensor(test_label)

In [28]:
model = tf.keras.Sequential(
    [
       tf.keras.Input(shape=(214,)),
       tf.keras.layers.Dense(25, activation="leaky_relu", name="L1"),
       tf.keras.layers.Dropout(0.2),
       tf.keras.layers.Dense(25, activation="leaky_relu", name="L3"),
       tf.keras.layers.Dropout(0.2),
       tf.keras.layers.Dense(10, activation="leaky_relu", name="L5"),
       tf.keras.layers.Dropout(0.2),
       tf.keras.layers.Dense(5, activation="leaky_relu", name="L7"),
       tf.keras.layers.Dropout(0.2),
       tf.keras.layers.Dense(1, activation="sigmoid", name="L9") 
    ]
)

In [29]:
from tensorflow.keras import backend as K

# Sensitivity (Recall)
import tensorflow as tf
from tensorflow.keras import backend as K

def sensitivity(y_true, y_pred):
    """
    param:
    y_pred - Predicted labels
    y_true - True labels 
    Returns:
    Specificity score
    """
    pos_y_true = y_true
    pos_y_pred = y_pred
    fn = K.sum(pos_y_true * y_pred)
    tp = K.sum(pos_y_true * pos_y_pred)
    specificity = tp / (tp + fn + K.epsilon())
    
    return sensitivity
# Specificity
def specificity(y_true, y_pred):
    """
    param:
    y_pred - Predicted labels
    y_true - True labels 
    Returns:
    Specificity score
    """
    neg_y_true = 1 - y_true
    neg_y_pred = 1 - y_pred
    fp = K.sum(neg_y_true * y_pred)
    tn = K.sum(neg_y_true * neg_y_pred)
    specificity = tn / (tn + fp + K.epsilon())
    spec_val = specificity
    return specificity

In [30]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-3),
    loss=tf.keras.losses.BinaryCrossentropy(), 
    metrics=["accuracy", specificity]
)

In [ ]:
history = model.fit(train_tf, train_label_tf, epochs=1000, validation_data = (dev_tf, dev_label_tf))

In [ ]:
loss_values = history.history['loss']
val_loss_values = history.history.get('val_loss')

# Plot the loss values
plt.figure(figsize=(10, 6))
plt.plot(loss_values, label='Training Loss')
if val_loss_values:
    plt.plot(val_loss_values, label='Validation Loss', linestyle='--')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model.evaluate(dev_tf, dev_label_tf)